In [2]:
import omdb
import sqlite3
import pandas as pd
import os
import requests
import shutil
import io
import PIL.Image as Image

# first read in OMDB python key
with open('api_key') as f:
    lines = f.readlines()
api_key = lines[0]
omdb.set_default('apikey', api_key)


In [3]:
# set acronyms
st = {'tos': {'title': 'The Original Series', 'imdbid': 'tt0060028'},
      'tas': {'title': 'The Animated Series', 'imdbid': 'tt0069637'},
      'tng': {'title': 'The Next Generation', 'imdbid': 'tt0092455'},
      'ds9': {'title': 'Deep Space Nine',     'imdbid': 'tt0106145'},
      'voy': {'title': 'Voyager',             'imdbid': 'tt0112178'},
      'ent': {'title': 'Enterprise',          'imdbid': 'tt0244365'},
      'dis': {'title': 'Discovery',           'imdbid': 'tt5171438'},
      'sho': {'title': 'Short Treks',         'imdbid': 'tt9059594'},
      'pic': {'title': 'Picard',              'imdbid': 'tt8806524'},
      'lds': {'title': 'Lower Decks',         'imdbid': 'tt9184820'},
      'pro': {'title': 'Prodigy',             'imdbid': 'tt9795876'},
      'snw': {'title': 'Strange New Worlds',  'imdbid': 'tt12327578'},
      } 

## Series Request

In [6]:
db_filename = "treklist.db"

# delete current database
os.remove(db_filename)

# make omdb requests for series
conn = sqlite3.connect(db_filename)
curs = conn.cursor()

conn.execute("DROP TABLE IF EXISTS series")
conn.execute("CREATE TABLE series (abb TEXT, title TEXT, imdb_id TEXT, year TEXT, total_seasons INTEGER, poster_url TEXT, poster BLOB, rated TEXT)")
for key in st.keys():
    res = omdb.imdbid(st[key]['imdbid'])
    img = requests.get(res['poster'], stream=True)

    cmd = "INSERT INTO series (abb, title, imdb_id, year, total_seasons, poster_url, rated) VALUES (?, ?, ?, ?, ?, ?, ?) "
    curs.execute(cmd, [key, res['title'], res['imdb_id'], res['year'], res['total_seasons'], res['poster'], res['rated']])
    conn.commit()

df = pd.read_sql_query("SELECT * FROM series", conn)
df



    abb                           title     imdb_id       year  total_seasons  \
0   tos                       Star Trek   tt0060028  1966–1969              3   
1   tas  Star Trek: The Animated Series   tt0069637  1973–1975              2   
2   tng  Star Trek: The Next Generation   tt0092455  1987–1994              7   
3   ds9      Star Trek: Deep Space Nine   tt0106145  1993–1999              7   
4   voy              Star Trek: Voyager   tt0112178  1995–2001              7   
5   ent           Star Trek: Enterprise   tt0244365  2001–2005              4   
6   dis            Star Trek: Discovery   tt5171438      2017–              5   
7   sho          Star Trek: Short Treks   tt9059594      2018–              2   
8   pic               Star Trek: Picard   tt8806524      2020–              3   
9   lds          Star Trek: Lower Decks   tt9184820      2020–              3   
10  pro              Star Trek: Prodigy   tt9795876      2021–              1   
11  snw   Star Trek: Strange

## Episodes Request

In [23]:
keys = list(st.keys())
for key in keys: # temporary for testing
    conn.execute(f"CREATE TABLE IF NOT EXISTS {key} (title TEXT, rated TEXT, released DATE, season INTEGER, episode INTEGER, runtime TEXT, director TEXT, writer TEXT, actors TEXT, plot TEXT, poster_url TEXT, poster BLOB, imdb_rating FLOAT, imdb_votes INTEGER, imdb_id TEXT)")

    tot_seasons = df['total_seasons'][df['abb'] == key].values[0]
    print('key %s seasons %i' % (key, tot_seasons))
    for season in range(tot_seasons):# another temporary....
        season += 1 # account for zero index
        res = omdb.imdbid(st[key]['imdbid'], season=season)
        for ep in res['episodes']:

            # check if it is already in the database
            df_check = pd.read_sql_query(f"SELECT * FROM {key} WHERE imdb_id = '{ep['imdb_id']}'", conn)
            ep_exists = True if len(df_check) > 0 else False
            if not ep_exists:
                ep_info = omdb.imdbid(ep['imdb_id'])
                cmd = f"INSERT INTO {key} (title, rated, released, season, episode, runtime, director, writer, actors, plot, poster_url, imdb_rating, imdb_votes, imdb_id) VALUES " + "(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
                curs.execute(cmd, [ep_info['title'], ep_info['rated'], ep['released'], season, ep_info['episode'], ep_info['runtime'], ep_info['director'], ep_info['writer'], ep_info['actors'], ep_info['plot'], ep_info['poster'], ep_info['imdb_rating'], ep_info['imdb_votes'], ep_info['imdb_id']])
    conn.commit()

key tos seasons 3
key tas seasons 2
key tng seasons 7
key ds9 seasons 7
key voy seasons 7
key ent seasons 4
key dis seasons 5
key sho seasons 2
key pic seasons 3
key lds seasons 3
key pro seasons 1
key snw seasons 1


## Download/Insert Posters

In [45]:
# Download posters for series only
df = pd.read_sql_query(f"SELECT * from series", conn)
for i, row in df.iterrows():
    poster_url = row['poster_url']
    if poster_url != "N/A":
        # insert poster
        img = requests.get(row['poster_url'], stream=True)
        cmd = f"UPDATE series SET poster = ? WHERE imdb_id = '{row['imdb_id']}'"
        curs.execute(cmd, [sqlite3.Binary(img.content)])
        conn.commit()

In [62]:
# optional - read back a poster
curs.execute("SELECT * FROM tos WHERE imdb_id = 'tt0708469'")
record = curs.fetchall()
image = Image.open(io.BytesIO(record[0][6]))
image.show()

TypeError: a bytes-like object is required, not 'str'

In [59]:
# download poster for individual series
key = keys[0]
df = pd.read_sql_query(f"SELECT * from {key}", conn)
for i, row in df.iterrows():
    poster_url = row['poster_url']
    if poster_url != "N/A":
        # insert poster
        img = requests.get(row['poster_url'], stream=True)
        cmd = f"UPDATE {key} SET poster = ? WHERE imdb_id = '{row['imdb_id']}'"
        curs.execute(cmd, [sqlite3.Binary(img.content)])
        conn.commit()

In [67]:
# optional - read back a poster
record = curs.execute("SELECT * FROM tos WHERE imdb_id = 'tt0708415'")
print(record)
record = curs.fetchall()
image = Image.open(io.BytesIO(record[0][11]))
image.show()

In [61]:
df = pd.read_sql_query(f"SELECT * from tos", conn)
df

,title,rated,released,season,episode,runtime,director,writer,actors,plot,poster_url,poster,imdb_rating,imdb_votes,imdb_id
0,The Man Trap,TV-PG,1966-09-08,1,1,50 min,Marc Daniels,"George Clayton Johnson, Gene Roddenberry","William Shatner, Leonard Nimoy, Jeanne Bal",Dr. McCoy discovers his old flame is not what ...,https://m.media-amazon.com/images/M/MV5BNDdiNG...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,7.2,4853,tt0708469
1,Charlie X,TV-PG,1966-09-15,1,2,50 min,Lawrence Dobkin,"D.C. Fontana, Gene Roddenberry","William Shatner, Leonard Nimoy, Robert Walker Jr.",Captain Kirk must learn the limits to the powe...,https://m.media-amazon.com/images/M/MV5BZDdlY2...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,7.1,4425,tt0708424
2,Where No Man Has Gone Before,TV-PG,1966-09-22,1,3,50 min,James Goldstone,"Samuel A. Peeples, Gene Roddenberry","William Shatner, Leonard Nimoy, Gary Lockwood",The flight recorder of the 200-year-old U.S.S....,https://m.media-amazon.com/images/M/MV5BOGIzMD...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,N/A,5431,tt0061027
3,The Naked Time,TV-PG,1966-09-29,1,4,50 min,Marc Daniels,"John D.F. Black, Gene Roddenberry","William Shatner, Leonard Nimoy, Stewart Moss",The crew is infected with a mysterious disease...,https://m.media-amazon.com/images/M/MV5BMjdjNj...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,7.8,4278,tt0708473
4,The Enemy Within,TV-PG,1966-10-06,1,5,50 min,Leo Penn,"Richard Matheson, Gene Roddenberry","William Shatner, Leonard Nimoy, DeForest Kelley",A transporter malfunction splits Captain Kirk ...,https://m.media-amazon.com/images/M/MV5BZWQ2ZT...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,7.6,4055,tt0708463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,The Way to Eden,TV-PG,1969-02-21,3,20,51 min,David Alexander,"Gene Roddenberry, Arthur Heinemann, D.C. Fontana","William Shatner, Leonard Nimoy, DeForest Kelley","A group of idealistic hippies, led by an irrat...",https://m.media-amazon.com/images/M/MV5BMDM1Mj...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,5.4,2773,tt0708482
72,The Cloud Minders,TV-PG,1969-02-28,3,21,51 min,Jud Taylor,"Gene Roddenberry, Margaret Armen, David Gerrold","William Shatner, Leonard Nimoy, DeForest Kelley",Kirk and Spock are caught up in a revolution o...,https://m.media-amazon.com/images/M/MV5BMjU5Ym...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,7.0,2491,tt0708456
73,The Savage Curtain,TV-PG,1969-03-07,3,22,51 min,Herschel Daugherty,"Gene Roddenberry, Arthur Heinemann, Arthur H. ...","William Shatner, Leonard Nimoy, DeForest Kelley","Kirk, Spock, Abraham Lincoln and Vulcan legend...",https://m.media-amazon.com/images/M/MV5BNmZjM2...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,N/A,2521,tt0708477
74,All Our Yesterdays,TV-PG,1969-03-14,3,23,50 min,Marvin J. Chomsky,"Gene Roddenberry, Jean Lisette Aroeste, Arthur...","William Shatner, Leonard Nimoy, DeForest Kelley","When Kirk, Spock and McCoy investigate the dis...",https://m.media-amazon.com/images/M/MV5BNDY3OT...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,8.1,2766,tt0708415
